In [37]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("imdb")

Reusing dataset imdb (C:\Users\I0466996\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [38]:
del raw_datasets["unsupervised"]
raw_datasets = raw_datasets.shuffle(seed=42)
raw_datasets

Loading cached shuffled indices for dataset at C:\Users\I0466996\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-f700cd6f222bd13f.arrow
Loading cached shuffled indices for dataset at C:\Users\I0466996\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-b23cfeb68a931a8d.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
})

In [39]:
raw_datasets["train"]["text"][1:2]

['This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stub your toe on the moon" It reminds me of Sinatra\'s song High Hopes, it is fun and inspirational. The Music is great throughout and my favorite song is sung by the King, Hank (bing Crosby) and Sir "Saggy" Sagamore. OVerall a great family movie or even a great Date movie. This is a movie you can watch over and over again. The princess played by Rhonda Fleming is gorgeous. I love this movie!! If you liked Danny Kaye in the Court Jester then you will definitely like this movie.']

In [40]:
raw_datasets["train"].shape[0]

25000

In [41]:
num_clients = 3
trainset = raw_datasets["train"]

In [42]:
part_num = int(trainset.data.shape[0] / num_clients)
last_n = trainset.data.shape[0] - (part_num*2)

In [43]:
split_n = [part_num] * (num_clients-1)
split_n.append(last_n)

In [44]:
import torch.utils.data

traindata_split = torch.utils.data.random_split(trainset, split_n)

In [45]:
from pathlib import Path

path = Path('./split_indices/').expanduser()
path.mkdir(parents=True, exist_ok=True)

prefix = "split_part"

for i in range(len(traindata_split)):
    print("==> saving part " + str(i+1) + " out of", len(traindata_split))
    torch.save(traindata_split[i], path/(prefix+str(i)+'.pt'))

==> saving part 1 out of 3
==> saving part 2 out of 3
==> saving part 3 out of 3


In [46]:
import numpy as np

print("==> asserting part 0 correctness.")
check_tensor = torch.load(path/(prefix+str(0)+'.pt'))
dataset1 = torch.utils.data.dataset.Subset(trainset, check_tensor.indices)
dataset2 = torch.utils.data.dataset.Subset(trainset, traindata_split[0].indices)
if all(np.equal(dataset1.indices, dataset2.indices)):
    print("==> assert OK.")
else:
    print("==> assert is NOT OK.")

==> asserting part 0 correctness.
==> assert OK.
